Code adapted from  https://shephexd.github.io/development/2017/02/19/pycuda.html

If using google colab:
* Click on Runtime (excecution) and select Change runtime type (modifier le type d'excecution).
  Then select GPU in Hardware Acceleration (accélérateur matériel)
* Start your session by installing pycuda with the command:

  -> !pip install pycuda

In [3]:
!pip install pycuda

     |████████████████████████████████| 1.6MB 9.1MB/s 
     |████████████████████████████████| 71kB 8.2MB/s 
     |████████████████████████████████| 81kB 7.2MB/s 
  Created wheel for pycuda: filename=pycuda-2020.1-cp36-cp36m-linux_x86_64.whl size=621085 sha256=ca3286ca8dda7977dddd4bd47ed84023ce90caff2d7f7fb3c21630723be1608c
  Stored in directory: /root/.cache/pip/wheels/8f/78/d1/5bb826f81d9d490297a348d818ff3ee6dd6f2075b06dde6ea0
  Created wheel for pytools: filename=pytools-2020.4.3-py2.py3-none-any.whl size=61374 sha256=093d7f11c0d41991632aad053c6c4fff000b062483f326ffbb3dc51f3eeb0d25
  Stored in directory: /root/.cache/pip/wheels/af/c7/81/a22edb90b0b09a880468b2253bb1df8e9f503337ee15432c64
Successfully built pycuda pytools


In [4]:
import numpy as np
from pycuda import driver, compiler, gpuarray, tools
import time

In [5]:
# -- initialize the device
import pycuda.autoinit


#get device information
MyDevice=pycuda.driver.Device(0)
MyDevice.get_attributes()

{pycuda._driver.device_attribute.MAX_THREADS_PER_BLOCK: 1024,
 pycuda._driver.device_attribute.MAX_BLOCK_DIM_X: 1024,
 pycuda._driver.device_attribute.MAX_BLOCK_DIM_Y: 1024,
 pycuda._driver.device_attribute.MAX_BLOCK_DIM_Z: 64,
 pycuda._driver.device_attribute.MAX_GRID_DIM_X: 2147483647,
 pycuda._driver.device_attribute.MAX_GRID_DIM_Y: 65535,
 pycuda._driver.device_attribute.MAX_GRID_DIM_Z: 65535,
 pycuda._driver.device_attribute.MAX_SHARED_MEMORY_PER_BLOCK: 49152,
 pycuda._driver.device_attribute.TOTAL_CONSTANT_MEMORY: 65536,
 pycuda._driver.device_attribute.WARP_SIZE: 32,
 pycuda._driver.device_attribute.MAX_PITCH: 2147483647,
 pycuda._driver.device_attribute.MAX_REGISTERS_PER_BLOCK: 65536,
 pycuda._driver.device_attribute.CLOCK_RATE: 1590000,
 pycuda._driver.device_attribute.TEXTURE_ALIGNMENT: 512,
 pycuda._driver.device_attribute.GPU_OVERLAP: 1,
 pycuda._driver.device_attribute.MULTIPROCESSOR_COUNT: 40,
 pycuda._driver.device_attribute.KERNEL_EXEC_TIMEOUT: 0,
 pycuda._driver.device

In [6]:
#define the kernel
kernel_code_template = """
__global__ void MatrixMulKernel(float *a, float *b, float *c)
{
    int tx = threadIdx.x;
    int ty = threadIdx.y;

    // Pvalue is used to store the element of the matrix
    // that is computed by the thread
    float Pvalue = 0;

    // Each thread loads one row of M and one column of N,
    //   to produce one element of P.
    for (int k = 0; k < %(MATRIX_SIZE)s; ++k) {
        float Aelement = a[ty * %(MATRIX_SIZE)s + k];
        float Belement = b[k * %(MATRIX_SIZE)s + tx];
        Pvalue += Aelement * Belement;
    }

    // Write the matrix to device memory;
    // each thread writes one element
    c[ty * %(MATRIX_SIZE)s + tx] = Pvalue;
}
"""

In [30]:
# define the (square) matrix size
#  note that we'll only use *one* block of threads here
#  as a consequence this number (squared) can't exceed max_threads
# -> use MyDevice.get_attributes() to get this information
MATRIX_SIZE = 32

# create two random square matrices
a_cpu = np.random.randn(MATRIX_SIZE, MATRIX_SIZE).astype(np.float32)
b_cpu = np.random.randn(MATRIX_SIZE, MATRIX_SIZE).astype(np.float32)

In [31]:
# compute reference on the CPU to verify GPU computation
time_start=time.time()
c_cpu = np.dot(a_cpu, b_cpu)
time_end=time.time()
timeCPU = time_end-time_start
print('enlapsed time (CPU):',timeCPU,' seconds')

enlapsed time (CPU): 0.00013875961303710938  seconds


In [32]:
# transfer host (CPU) memory to device (GPU) memory
a_gpu = gpuarray.to_gpu(a_cpu)
b_gpu = gpuarray.to_gpu(b_cpu)

# create empty gpu array for the result (C = A * B)
c_gpu = gpuarray.empty((MATRIX_SIZE, MATRIX_SIZE), np.float32)

# get the kernel code from the template
# by specifying the constant MATRIX_SIZE
kernel_code = kernel_code_template % {
    'MATRIX_SIZE': MATRIX_SIZE
    }

# compile the kernel code
mod = compiler.SourceModule(kernel_code)

# get the kernel function from the compiled module
matrixmul = mod.get_function("MatrixMulKernel")

# call the kernel on the card
time_start=time.time()

matrixmul(
    # inputs
    a_gpu, b_gpu,
    # output
    c_gpu,
    # (only one) block of MATRIX_SIZE x MATRIX_SIZE threads
    block = (MATRIX_SIZE, MATRIX_SIZE, 1),
    )

time_end=time.time()
timeGPU = time_end-time_start
print('enlapsed time (GPU):',timeGPU,' seconds')

enlapsed time (GPU): 0.0001888275146484375  seconds


In [33]:
# print the results
def display(verbose=False):

  print("Taille matrices : ", MATRIX_SIZE)
  #print Matrices
  if verbose == True:
    print("-" * 80)
    print("Matrix A (GPU):")
    print(a_gpu.get())

    print("-" * 80)
    print("Matrix B (GPU):")
    print(b_gpu.get())

    print("-" * 80)
    print("Matrix C (GPU):")
    print(c_gpu.get())

    print("-" * 80)

  #print difference
  print("CPU-GPU difference:")
  norm = np.linalg.norm(c_cpu - c_gpu.get())
  if norm != 0:
    print(c_cpu - c_gpu.get())
  else:
    print(norm)

  #print difference time
  print()
  print("Rapport de temps CPU/GPU : ", round(timeCPU/timeGPU,3))

display()

Taille matrices :  32
CPU-GPU difference:
0.0

Rapport de temps CPU/GPU :  0.735


#QUESTION 1: Comprenez bien chaque partie du code

Done

#QUESTION 2: Comparez le temps necessaire pour la multiplication en CPU et en GPU pour MATRIX_SIZE = 8, 16, 32. Qu'en pensez vous?

Le calcul avec CPU est plus rapide. Cela vient sûrement du fait qu'avec des matrices de cette taille, on perd plus de temps à transférer les données d'une zone mémoire à une autre que le temps gagné avec la parallélisation.

#QUESTION 3: Par quelle methode simple pourriez vous rendre la parallelisation GPU competitive par rapport a la methode CPU (i.e. avec numpy)?

Aucune idée